# Téléchargement des Datasets - Approche Progressive

**Objectif :** Télécharger les datasets pour modèle prédictif électoral **évolutif**

**Stratégie d'extensibilité :**
- **Phase 1** : POC Petite Couronne (150 communes) - Datasets socio-économiques (~2.4 GB)
- **Phase 2** : Extension France entière (35K communes) - Enrichissement territorial (~140 MB)

**Périmètre initial :** Petite Couronne (75, 92, 93, 94)  
**Périmètre cible :** France entière - Architecture Big Data scalable

---

## Configuration

In [17]:
import requests
import os
from pathlib import Path

# Configuration des chemins
DATA_RAW = Path("/app/data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

# Départements cibles
DEPARTEMENTS = ['75', '92', '93', '94']

print(f"✅ Dossier de destination : {DATA_RAW}")
print(f"📍 Périmètre : {', '.join(DEPARTEMENTS)}")

✅ Dossier de destination : /app/data/raw
📍 Périmètre : 75, 92, 93, 94


## Fonction utilitaire de téléchargement

In [18]:
def download_file(url, filename, force=False):
    """
    Télécharge un fichier depuis une URL
    
    Args:
        url: URL du fichier
        filename: Nom du fichier de destination
        force: Forcer le téléchargement même si le fichier existe
    """
    filepath = DATA_RAW / filename
    
    if filepath.exists() and not force:
        print(f"⏭️  {filename} existe déjà")
        return filepath
    
    print(f"⬇️  Téléchargement de {filename}...")
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        
        with open(filepath, 'wb') as f:
            f.write(response.content)
        
        size_mb = filepath.stat().st_size / (1024 * 1024)
        print(f"✅ {filename} téléchargé ({size_mb:.2f} MB)")
        return filepath
        
    except Exception as e:
        print(f"❌ Erreur lors du téléchargement de {filename}: {e}")
        return None

print("✅ Fonction de téléchargement prête")

✅ Fonction de téléchargement prête


## 🎯 PHASE 1 - POC Petite Couronne (Datasets Essentiels)

**Objectif** : Modèle de base avec indicateurs socio-économiques classiques  
**Périmètre** : 150 communes (75, 92, 93, 94)  
**Taille totale** : ~2.4 GB

### 1.1 Élections agrégées 1999-2024 ✨ VARIABLE CIBLE

In [19]:
# Élections agrégées - Toutes élections 1999-2024 (2.2 Go)
# Contient : Présidentielles, Législatives, Européennes, Régionales, Départementales, Municipales
# URL directe pour téléchargement
url_elections_agregees = "https://object.files.data.gouv.fr/data-pipeline-open/elections/candidats_results.csv"
download_file(url_elections_agregees, "elections_agregees_1999_2024.csv")

print("\n⚠️  Ce fichier est volumineux (2.2 Go) - Le téléchargement peut prendre plusieurs minutes")

⏭️  elections_agregees_1999_2024.csv existe déjà

⚠️  Ce fichier est volumineux (2.2 Go) - Le téléchargement peut prendre plusieurs minutes


### 1.2 Revenus par commune

In [20]:
# Revenus des Français par commune (4.8 MB)
url_revenus = "https://static.data.gouv.fr/resources/revenu-des-francais-a-la-commune/20251210-134014/revenu-des-francais-a-la-commune-1765372688826.csv"
download_file(url_revenus, "revenus_commune.csv")

⏭️  revenus_commune.csv existe déjà


PosixPath('/app/data/raw/revenus_commune.csv')

### 1.3 Population historique 1968-2022

*Dynamique démographique - Croissance/déclin urbain*

In [21]:
# Population des 16 ans ou plus de 1968 à 2022 (40 Mo)
url_pop_historique = "https://www.insee.fr/fr/statistiques/fichier/1893149/pop-16ans-dipl6822.zip"
download_file(url_pop_historique, "population_historique_1968_2022.zip")

print("\n⚠️  Fichier ZIP téléchargé - L'extraction automatique sera faite plus bas")

⏭️  population_historique_1968_2022.zip existe déjà

⚠️  Fichier ZIP téléchargé - L'extraction automatique sera faite plus bas


### 1.4 Diplômes et Formation 2022

*Niveau d'éducation - Prédicteur majeur du vote (recherches sociologie électorale)*

In [22]:
# Diplômes et formation par commune (ZIP contenant CSV 81 MB)
url_diplomes = "https://www.insee.fr/fr/statistiques/fichier/8581488/base-cc-diplomes-formation-2022_xlsx.zip"
download_file(url_diplomes, "diplomes_formation_2022.zip")

print("\n⚠️  Fichier ZIP - Extraction automatique plus bas")

⏭️  diplomes_formation_2022.zip existe déjà

⚠️  Fichier ZIP - Extraction automatique plus bas


### 1.5 CSP des actifs 25-54 ans

*Catégories socio-professionnelles - Vote de classe (ouvriers ≠ cadres)*

In [23]:
# CSP des actifs par commune (ZIP contenant XLSX 28.5 MB)
url_csp = "https://www.insee.fr/fr/statistiques/fichier/1893185/pop-act2554-csp-cd-6822.zip"
download_file(url_csp, "csp_actifs_2554.zip")

print("\n✅ Phase 1 terminée - Modèle de base prêt pour Petite Couronne")
print("💡 Ces 5 datasets permettent un modèle robuste avec les 3 variables socio-éco de référence")

⏭️  csp_actifs_2554.zip existe déjà

✅ Phase 1 terminée - Modèle de base prêt pour Petite Couronne
💡 Ces 5 datasets permettent un modèle robuste avec les 3 variables socio-éco de référence


## 📊 PHASE 2 - Extension France Entière (Enrichissement Territorial)

**Objectif** : Capturer la diversité territoriale française (urbain/rural, littoral/montagne)  
**Périmètre** : 35 000 communes  
**Taille totale** : ~140 MB  
**Apport** : Contexte local (finances, environnement, démographie fine)

### 2.1 Comptes individuels des communes 2022

*Santé financière locale - Investissements publics, endettement*  
*Pertinent pour communes rurales (diversité financière) vs Petite Couronne (homogène)*

In [24]:
# Comptes communaux 2022 (50 MB)
print("⚠️  Comptes communaux : Téléchargement manuel requis")
print("   Lien : https://data.economie.gouv.fr/explore/dataset/comptes-individuels-des-communes")
print("   Rechercher 'comptes individuels communes 2022' et exporter CSV")
print("\n💡 Surtout utile pour extension France entière (diversité finances locales)")

⚠️  Comptes communaux : Téléchargement manuel requis
   Lien : https://data.economie.gouv.fr/explore/dataset/comptes-individuels-des-communes
   Rechercher 'comptes individuels communes 2022' et exporter CSV

💡 Surtout utile pour extension France entière (diversité finances locales)


### 2.2 Catastrophes naturelles et Risques (GASPAR)

*Exposition aux risques environnementaux - Impact sur vote écologique/local*  
*Peu pertinent Petite Couronne, TRÈS pertinent France entière (littoral, montagne, zones rurales)*

In [25]:
# Catastrophes naturelles par commune (34.5 MB)
url_catnat = "https://www.data.gouv.fr/fr/datasets/r/4c176fa2-b0cd-4780-b644-f34cecab89fb"
download_file(url_catnat, "catastrophes_naturelles_gaspar.csv")

# Risques connus par commune
url_risques = "https://www.data.gouv.fr/fr/datasets/r/696f89e0-5d1d-4005-afd8-b42c9e2b5a5e"
download_file(url_risques, "risques_gaspar.csv")

print("\n💡 Capture diversité territoriale : inondations, sécheresses, érosion côtière, avalanches")

⬇️  Téléchargement de catastrophes_naturelles_gaspar.csv...
❌ Erreur lors du téléchargement de catastrophes_naturelles_gaspar.csv: 404 Client Error: NOT FOUND for url: https://www.data.gouv.fr/api/1/datasets/r/4c176fa2-b0cd-4780-b644-f34cecab89fb
⬇️  Téléchargement de risques_gaspar.csv...
❌ Erreur lors du téléchargement de risques_gaspar.csv: 404 Client Error: NOT FOUND for url: https://www.data.gouv.fr/api/1/datasets/r/696f89e0-5d1d-4005-afd8-b42c9e2b5a5e

💡 Capture diversité territoriale : inondations, sécheresses, érosion côtière, avalanches


### 2.3 Démographie dynamique fine (Naissances/Décès)

*Vieillissement vs rajeunissement - Solde naturel*  
*Enrichit données population : communes retraités ≠ communes jeunes familles*

In [26]:
# Naissances par commune 2008-2024 (24.4 MB)
print("ℹ️  Naissances : https://www.insee.fr/fr/statistiques/1893255")

# Décès par commune 2008-2024 (24.4 MB)
print("ℹ️  Décès : https://www.insee.fr/fr/statistiques/1893253")

print("\n💡 Optionnel mais affine modèle France entière (diversité démographique)")
print("✅ Phase 2 terminée - Enrichissement territorial pour extension scalable")

ℹ️  Naissances : https://www.insee.fr/fr/statistiques/1893255
ℹ️  Décès : https://www.insee.fr/fr/statistiques/1893253

💡 Optionnel mais affine modèle France entière (diversité démographique)
✅ Phase 2 terminée - Enrichissement territorial pour extension scalable


## 🗺️ Référentiel Géographique (Transversal)

In [27]:
# Code Officiel Géographique (COG) - Liste des communes
url_communes = "https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv"
download_file(url_communes, "referentiel_communes.csv")

print("\n💡 Clé de jointure pour TOUS les datasets - Codes INSEE, départements, régions")

⏭️  referentiel_communes.csv existe déjà

💡 Clé de jointure pour TOUS les datasets - Codes INSEE, départements, régions


In [28]:
import zipfile

def extract_zip_files():
    """
    Extrait automatiquement tous les fichiers ZIP dans data/raw/
    et supprime les archives après extraction
    """
    zip_files = list(DATA_RAW.glob('*.zip'))
    
    if not zip_files:
        print("ℹ️  Aucun fichier ZIP à extraire")
        return
    
    print(f"\n📦 Extraction de {len(zip_files)} fichier(s) ZIP...\n")
    
    for zip_path in zip_files:
        try:
            # Créer un dossier avec le nom du fichier (sans .zip)
            extract_dir = DATA_RAW / zip_path.stem
            extract_dir.mkdir(exist_ok=True)
            
            print(f"⬇️  Extraction de {zip_path.name}...")
            
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
            
            # Lister les fichiers extraits
            extracted_files = list(extract_dir.glob('*'))
            print(f"   ✅ {len(extracted_files)} fichier(s) extrait(s) dans {extract_dir.name}/")
            
            for extracted_file in extracted_files[:3]:  # Afficher les 3 premiers
                size_mb = extracted_file.stat().st_size / (1024 * 1024)
                print(f"      • {extracted_file.name} ({size_mb:.2f} MB)")
            
            if len(extracted_files) > 3:
                print(f"      ... et {len(extracted_files) - 3} autre(s) fichier(s)")
            
            # Supprimer le fichier ZIP après extraction réussie
            zip_path.unlink()
            print(f"   🗑️  {zip_path.name} supprimé")
                
        except Exception as e:
            print(f"   ❌ Erreur lors de l'extraction de {zip_path.name}: {e}")
    
    print("\n✅ Extraction terminée et fichiers ZIP supprimés !")

# Lancer l'extraction
extract_zip_files()


📦 Extraction de 3 fichier(s) ZIP...

⬇️  Extraction de diplomes_formation_2022.zip...
   ✅ 1 fichier(s) extrait(s) dans diplomes_formation_2022/
      • base-cc-diplomes-formation-2022.xlsx (66.06 MB)
   🗑️  diplomes_formation_2022.zip supprimé
⬇️  Extraction de population_historique_1968_2022.zip...
   ✅ 1 fichier(s) extrait(s) dans population_historique_1968_2022/
      • pop-16ans-dipl6822.xlsx (43.63 MB)
   🗑️  population_historique_1968_2022.zip supprimé
⬇️  Extraction de csp_actifs_2554.zip...
   ✅ 1 fichier(s) extrait(s) dans csp_actifs_2554/
      • pop-act2554-csp-cd-6822.xlsx (28.53 MB)
   🗑️  csp_actifs_2554.zip supprimé

✅ Extraction terminée et fichiers ZIP supprimés !


## 📦 Extraction automatique des fichiers ZIP

## 6. Vérification des fichiers téléchargés

In [29]:
# Liste tous les fichiers téléchargés
print("\n📦 Fichiers téléchargés dans data/raw/ :\n")

files = sorted(DATA_RAW.glob('*'))
total_size = 0

for file in files:
    if file.is_file():
        size_mb = file.stat().st_size / (1024 * 1024)
        total_size += size_mb
        print(f"  • {file.name:<40} {size_mb:>8.2f} MB")

print(f"\n✅ Total : {len(files)} fichiers, {total_size:.2f} MB")


📦 Fichiers téléchargés dans data/raw/ :

  • .gitkeep                                     0.00 MB
  • elections_agregees_1999_2024.csv          2244.21 MB
  • referentiel_communes.csv                     2.55 MB
  • revenus_commune.csv                          4.80 MB

✅ Total : 7 fichiers, 2251.56 MB
